In [1]:
from datetime import datetime, timedelta, timezone
import threading
import json
import pandas as pd
from alpaca.data.historical import StockHistoricalDataClient, OptionHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, OptionBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.enums import DataFeed, OptionsFeed
from alpaca.data.live import StockDataStream
from config import Config

from abc import ABC, abstractmethod


class DataClient(ABC):
    def __init__(self):
        pass

    @abstractmethod
    def get_history(self, symbols, n_days, interval):
        pass

    
class StockClient:

    def __init__(self, params = None):
        self.client = StockHistoricalDataClient(Config.ALPACA_KEY, Config.ALPACA_SECRET)

        
    def get_history(self,symbols, n_days, interval):
        """minute level day history for today"""

        day  = datetime.today()  - timedelta(days=n_days)
        start, end  = self.US_trading_hours(day)
        interval = interval.lower()
        if interval == 'minute':
            timeframe = TimeFrame.Minute
        elif interval == 'hour':
            timeframe = TimeFrame.Hour
        else: # default day
            timeframe = TimeFrame.Day

        req = StockBarsRequest(symbol_or_symbols=symbols,
                            timeframe=timeframe,
                              feed=DataFeed.IEX, start = start, end = datetime.now()) 
        
        return self.client.get_stock_bars(req).df

        

    def US_trading_hours(self, day = datetime.now(timezone.utc)):
        # Ensure day is timezone aware
        if day.tzinfo is None:
            day = day.replace(tzinfo=timezone.utc)
            
        start = datetime(day.year, day.month, day.day, 13, 30, tzinfo=timezone.utc)  # 9:30 AM ET
        end   = datetime(day.year, day.month, day.day, 20, 0, tzinfo=timezone.utc)
        return start, end

class OptionsClient():
    def __init__(self):
        self.client = OptionHistoricalDataClient(Config.ALPACA_KEY, Config.ALPACA_SECRET)
    def get_history(self,symbols, n_days, interval):
        """minute level day history for today"""

        day  = datetime.today()  - timedelta(days=n_days)
        start, end  = self.US_trading_hours(day)
        interval = interval.lower()
        if interval == 'minute':
            timeframe = TimeFrame.Minute
        elif interval == 'hour':
            timeframe = TimeFrame.Hour
        else: # default day
            timeframe = TimeFrame.Day

        req = OptionBarsRequest(symbol_or_symbols=symbols,
                            timeframe=timeframe,
                              feed=OptionsFeed.INDICATIVE, start = start, end = datetime.now(timezone.utc)- timedelta(minutes=16)) 
        
        return self.client.get_option_bars(req).df

In [7]:
client = StockClient()




data = client.get_history(['AAPL', 'MSFT', 'TSLA', 'NVDA', 'AMZN'], 1, 'minute')
data.reset_index().tail()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
1955,TSLA,2026-01-12 20:59:00+00:00,448.85,449.12,448.71,448.86,8239.0,209.0,448.936139
1956,TSLA,2026-01-12 21:00:00+00:00,449.01,449.01,449.01,449.01,63.0,1.0,449.010000
1957,TSLA,2026-01-12 21:01:00+00:00,448.89,448.89,448.89,448.89,40.0,1.0,448.890000
1958,TSLA,2026-01-12 21:07:00+00:00,449.07,449.07,449.07,449.07,63.0,1.0,449.070000
1959,TSLA,2026-01-12 21:55:00+00:00,448.47,448.60,448.47,448.60,143.0,4.0,448.535000


,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2025-12-18 14:30:00+00:00,273.61,273.615,272.125,272.510,16871.0,362.0,272.903061
1,AAPL,2025-12-18 14:31:00+00:00,272.63,272.755,271.250,271.305,15406.0,204.0,271.787050
2,AAPL,2025-12-18 14:32:00+00:00,271.20,271.690,270.410,270.460,17353.0,226.0,271.406642
3,AAPL,2025-12-18 14:33:00+00:00,270.36,271.000,270.180,270.365,5852.0,87.0,270.405758
4,AAPL,2025-12-18 14:34:00+00:00,270.32,270.380,269.850,269.870,14675.0,206.0,270.149681
...,...,...,...,...,...,...,...,...,...
1980,TSLA,2025-12-18 21:11:00+00:00,482.67,482.670,482.670,482.670,100.0,5.0,482.670000
1981,TSLA,2025-12-18 21:19:00+00:00,482.58,482.580,482.580,482.580,85.0,3.0,482.580000
1982,TSLA,2025-12-18 21:20:00+00:00,482.70,482.700,482.700,482.700,90.0,3.0,482.700000
1983,TSLA,2025-12-18 21:25:00+00:00,483.48,483.480,483.480,483.480,85.0,3.0,483.480000
